In [1]:
import sys
import glob

import pandas as pd
import matplotlib.pyplot as plt

from larch import io, xafs
from larch import Interpreter
mylarch = Interpreter()

In [2]:
def larch_xafs_normalization(filepath, _larch=None):
    """
    """
    try:
        del group
    except:
        None
    group = io.read_ascii(filepath, labels='col1, col2')
    group.path = filepath
    group.is_frozen = False
    group.datatype = 'xas'
    group.plot_xlabel = 'col1'
    group.plot_ylabel = 'col2/1.0'
    group.xdat = group.data[0, : ]
    group.ydat = group.data[1, : ] / 1.0
    group.yerr = 1.000000
    group.energy = group.xdat
    group.mu = group.ydat
    xafs.sort_xafs(group, overwrite=True, fix_repeats=True)
    filename = filepath.split('\\')[-1].replace(' ', '_').replace('-', 'n')
    group.filename = filename
    group.filenumber = filename.split('_Eta')[0]
    group.eta = filename.split('Eta_')[-1].split(' mV')[0]
    xafs.pre_edge(group, pre1=-70.00, pre2=-25.00, nvict=0.00, nnorm=1.00, norm1=40.00, norm2=115.00)
    group.norm_poly = 1.0*group.norm
    group.dnormde=group.dmude/group.edge_step
    xafs.autobk(group, rbkg=0.85, kweight=2)
    return group

## Scratch

In [39]:
import json
import time
import numpy as np

from collections import OrderedDict

from larch import Group
from larch.fitting import Parameter, isParameter
from larch.utils.jsonutils import encode4js, decode4js
from larch.utils.strutils import fix_varname

In [59]:
def dumfn(fname, *args, _larch=None):
    expr = getattr(_larch, 'this_expr', 'save(foo)')
    expr = expr.replace('\n', ' ').replace('\r', ' ')

    grouplist = _larch.symtable._sys.saverestore_groups[:]
    print(grouplist)
    buff = ["#Larch Save File: 1.0",
            "#save.date: %s" % time.strftime('%Y-%m-%d %H:%M:%S'),
            "#save.command: %s" % expr,
            "#save.nitems:  %i" % len(args)]

    names = []
    if expr.startswith('save('):
        print('ye')
        names = [a.strip() for a in expr[5:-1].split(',')]
    try:
        names.pop(0)
    except:
        pass
    if len(names) < len(args):
        names.extend(["_unknown_"]*(len(args) - len(names)))

    for name, arg in zip(names, args):
        buff.append("#=> %s" % name)
        buff.append(json.dumps(encode4js(arg, grouplist=grouplist)))
        buff.append("")
        with open(fname, "w") as fh:
            fh.write("\n".join(buff))

In [13]:
for g in mylarch.symtable._sys.saverestore_groups[:]:
    print(g, g[0].__name__)

(<class 'larch.xrf.roi.ROI'>, <class 'larch.xrf.mca.MCA'>) ROI
(<class 'larch.xafs.diffkk.diffKKGroup'>, <class 'larch.xafs.feffrunner.FeffRunner'>, <class 'larch.xafs.feffdat.FeffDatFile'>, <class 'larch.xafs.feffdat.FeffPathGroup'>, <class 'larch.xafs.feffit.TransformGroup'>, <class 'larch.xafs.feffit.FeffitDataSet'>) diffKKGroup


In [5]:
import larch
from larch import Group, Interpreter
from larch import io
    
mylarch = Interpreter()

x = [1, 2, 3, 4]
y = [0.5, 1, 1, 0.1]
z = ['a', 'b', 'c', 'd']
g = Group('mygroup', x=x, y=y, z=z)

io.save('test.lsav', g, _larch=mylarch)

AttributeError: 'tuple' object has no attribute '__name__'